<a href="https://colab.research.google.com/github/rht6226/BTP-DD/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import os.path
import csv
import glob
import tensorflow as tf
import h5py as h5py
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model

In [ ]:
import os
import cv2
from scipy.spatial import distance

In [ ]:
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

In [ ]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [ ]:
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

In [ ]:
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

In [ ]:
def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [ ]:
def getFrame(vidcap, sec):
    start = 180000
    vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
    hasFrames,image = vidcap.read()
    return hasFrames, image

In [ ]:
def extract_data_from_video(video_path, label):
    data = []
    labels = []
    vidcap = cv2.VideoCapture(video_path)
    sec = 0
    frameRate = 1
    success, image  = getFrame(vidcap, sec)
    count = 0
    misses = 0

    while success and count < 240: 
        landmarks = extract_face_landmarks(image)
        if sum(sum(landmarks)) != 0:
            count += 1
            data.append(landmarks)
            labels.append(label)
            sec = sec + frameRate
            sec = round(sec, 2)
            success, image = getFrame(vidcap, sec)

            if ((count - 1) %30) == 0:
                print("\n{}/240:".format(count), end='')
            print(">", end='')

        else:  
            sec = sec + frameRate
            sec = round(sec, 2)
            misses += 1
            success, image = getFrame(vidcap, sec)
            
            if ((misses) %30) == 0:
                print("\n{}/120:".format(misses), end='')
            print("x", end='')

            

        # break if no face in video
        if (misses > 120 and count <=5):
                break

    return data, labels

In [ ]:
def extract_data_from_folder(folderName):
    base_path = '/content/drive/MyDrive/Drowsiness Detection/Dataset/'
    folderPath = base_path + str(folderName) + '/'
    files = os.listdir(folderPath)
    data_all = []
    labels_all = []
    for f in files:
        print('\nExtracting data from {}/{}'.format(folderName, f))
        video_path = '/content/drive/MyDrive/Drowsiness Detection/Dataset/' + folderName + '/' + str(f)
        y = int(f.split('.')[0])
        data, labels = extract_data_from_video(video_path, y)
        data_all.append(data)
        labels_all.append(labels)
    return data_all, labels_all

In [ ]:
def extract_features_from_data(D):
    features = []
    for d in D:
        eye = d[36:68]
        ear = eye_aspect_ratio(eye)
        mar = mouth_aspect_ratio(eye)
        cir = circularity(eye)
        mouth_eye = mouth_over_eye(eye)
        features.append([ear, mar, cir, mouth_eye])
    features = np.array(features)
    return features

In [ ]:
def extraction(folderName):
    data, labels = extract_data_from_folder(folderName)
    D = []
    for d in data:
        for x in d:
            D.append(x)

    L = []

    for l in labels:
        for x in l:
            L.append(x)

    D = np.array(D)
    L = np.array(L)

    np.savez('/content/drive/MyDrive/Drowsiness Detection/Extracted_Data/{}.npz'.format(folderName), data=D, labels=L)

    F = extract_features_from_data(D)

    np.savez('/content/drive/MyDrive/Drowsiness Detection/Extracted_Features/{}.npz'.format(folderName), features=F, labels=L)

    print("\nExtraction for folder {} completed sucessfully...".format(folderName))
    print("Size of Facial Landmark Data : ", D.shape)
    print("Size of Extracted Features : ", F.shape)
    print("Size of Lables : ", L.shape)

In [ ]:
extraction("37")


Extracting data from 37/5.mov

1/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
31/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
61/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
91/240:>>>>>>>>>>>>>>>>>>>>>>>>

/usr/local/lib/python3.7/dist-packages/mlxtend/image/extract_face_landmarks.py:61: UserWarning: No face detected.
  warnings.warn('No face detected.')


x>>>>>>
121/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
151/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
181/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
211/240:>>>>>>>>>x>>>>>>>>>>>>>>>>>>>>>
Extracting data from 37/0.mov

1/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
31/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
61/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
91/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
121/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
151/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
181/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
211/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Extracting data from 37/10.mov
x
1/240:>>>xxxx>x>>xx>x>>>>>>>>>>>>>>>>>>>>>>>
31/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
61/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
91/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
121/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
151/240:>>>>>>>x>>x>>x>>>>>>>>>>>>>>>>>>>
181/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>x>>xxx>
211/240:>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Extraction for folder 37 completed sucessfully...
Size of Facial Landmark Data :  (720, 68, 2)
Size of Extracted Features :  (720, 4)
Size of 

In [ ]:
# All the features have been extracted into different files
# Now we will be combining them into one.

